In [3]:
import re
import pandas as pd

LOG_FILE = "training.log"   # path to your log file
OUTPUT_CSV = "is_street_metrics.csv"


def extract_is_street_metrics(log_path):
    results = []
    current_epoch = None

    # Regex patterns
    epoch_pattern = re.compile(r"performing (\d+)-th epoch")
    metric_pattern = re.compile(r"(val_)?is_street[_\w]*:\s*([0-9.eE+-]+)")

    with open(log_path, "r") as f:
        for line in f:
            # Detect epoch
            epoch_match = epoch_pattern.search(line)
            if epoch_match:
                current_epoch = int(epoch_match.group(1))

            # Extract metrics
            if "is_street" in line and current_epoch is not None:
                metrics = dict()
                metrics["epoch"] = current_epoch

                for match in metric_pattern.finditer(line):
                    metric_name = match.group(0).split(":")[0].strip()
                    metric_value = float(match.group(2))
                    metrics[metric_name] = metric_value

                # Only append if metrics were found
                if len(metrics) > 1:
                    results.append(metrics)

    return pd.DataFrame(results)


if __name__ == "__main__":
    df = extract_is_street_metrics(LOG_FILE)

    # Sort by epoch
    df = df.sort_values("epoch").reset_index(drop=True)

    print(df)
    df.to_csv(OUTPUT_CSV, index=False)

    print(f"\nSaved to {OUTPUT_CSV}")


    epoch  is_street__dice_coef  is_street_loss  is_street_pr_auc  \
0       0              0.010200          5.3204            0.0146   
1       1              0.003500          1.2151            0.0161   
2       2              0.000896          1.4939            0.0149   
3       3              0.008800          0.9465            0.0149   
4       4              0.010600          0.6304            0.0153   
..    ...                   ...             ...               ...   
90     90              0.027500          0.5160            0.0159   
91     91              0.025300          0.5296            0.0163   
92     92              0.023900          0.5232            0.0151   
93     93              0.027500          0.5176            0.0147   
94     94              0.032300          0.5135            0.0163   

    is_street_precision  is_street_recall  val_is_street__dice_coef  \
0                0.0148          0.108400              4.451600e-10   
1                0.0202      